In [1]:
import numpy as np
from typing import List, Tuple

class VEGA:
    """
    Vector Evaluated Genetic Algorithm (VEGA) implementation.
    This class implements VEGA for solving multi-objective optimization problems.
    """

    def __init__(self, pop_size: int, num_objectives: int, gene_size: int, 
                 crossover_rate: float, mutation_rate: float):
        """
        Initialize the VEGA.

        Args:
            pop_size (int): Size of the population.
            num_objectives (int): Number of objectives to optimize.
            gene_size (int): Number of genes in each individual.
            crossover_rate (float): Probability of crossover occurring.
            mutation_rate (float): Probability of mutation for each gene.
        """
        self.pop_size = pop_size
        self.num_objectives = num_objectives
        self.gene_size = gene_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.population = self.initialize_population()

    def initialize_population(self) -> np.ndarray:
        """
        Initialize a random population.

        Returns:
            np.ndarray: A 2D array where each row represents an individual.
        """
        return np.random.rand(self.pop_size, self.gene_size)

    def evaluate_fitness(self, individual: np.ndarray) -> List[float]:
        """
        Evaluate the fitness of an individual.
        This is a placeholder and should be replaced with actual objective functions.

        Args:
            individual (np.ndarray): The individual to evaluate.

        Returns:
            List[float]: A list of fitness values, one for each objective.
        """
        return [np.sum(individual) for _ in range(self.num_objectives)]

    def select_parents(self, subpopulation: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Select two parents from a subpopulation for reproduction.

        Args:
            subpopulation (np.ndarray): The subpopulation to select from.

        Returns:
            Tuple[np.ndarray, np.ndarray]: Two selected parent individuals.
        """
        idx1, idx2 = np.random.choice(len(subpopulation), 2, replace=False)
        return subpopulation[idx1], subpopulation[idx2]

    def crossover(self, parent1: np.ndarray, parent2: np.ndarray) -> np.ndarray:
        """
        Perform crossover between two parents.

        Args:
            parent1 (np.ndarray): First parent.
            parent2 (np.ndarray): Second parent.

        Returns:
            np.ndarray: Child produced by crossover.
        """
        if np.random.rand() < self.crossover_rate:
            crossover_point = np.random.randint(1, self.gene_size)
            child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        else:
            child = parent1.copy()
        return child

    def mutate(self, individual: np.ndarray) -> np.ndarray:
        """
        Perform mutation on an individual.

        Args:
            individual (np.ndarray): The individual to mutate.

        Returns:
            np.ndarray: The mutated individual.
        """
        mutation_mask = np.random.rand(self.gene_size) < self.mutation_rate
        individual[mutation_mask] = np.random.rand(np.sum(mutation_mask))
        return individual

    def run(self, generations: int) -> List[np.ndarray]:
        """
        Run the VEGA for a specified number of generations.

        Args:
            generations (int): Number of generations to run.

        Returns:
            List[np.ndarray]: The final Pareto front.
        """
        for _ in range(generations):
            fitness_values = np.array([self.evaluate_fitness(ind) for ind in self.population])
            new_population = []
            
            for obj in range(self.num_objectives):
                subpop_size = self.pop_size // self.num_objectives
                subpopulation = self.population[np.argsort(fitness_values[:, obj])[-subpop_size:]]
                
                for _ in range(subpop_size):
                    parent1, parent2 = self.select_parents(subpopulation)
                    child = self.crossover(parent1, parent2)
                    child = self.mutate(child)
                    new_population.append(child)
            
            self.population = np.array(new_population)
        
        return self.get_pareto_front()

    def get_pareto_front(self) -> List[np.ndarray]:
        """
        Get the Pareto front from the current population.
        This method is added to VEGA to extract Pareto-optimal solutions at the end of the run.

        Returns:
            List[np.ndarray]: The Pareto front (non-dominated solutions).
        """
        fitness_values = np.array([self.evaluate_fitness(ind) for ind in self.population])
        pareto_front = []
        for i, fitness in enumerate(fitness_values):
            if np.all(np.any(fitness_values >= fitness, axis=1)):
                pareto_front.append(self.population[i])
        return pareto_front

# Usage example
vega = VEGA(pop_size=100, num_objectives=2, gene_size=10, crossover_rate=0.8, mutation_rate=0.1)
pareto_front = vega.run(generations=50)


In [6]:
print(vega.initialize_population()[0])

[0.19264033 0.33327199 0.07978781 0.49665193 0.18044357 0.15914277
 0.1142943  0.79329286 0.39675815 0.92881043]
